## Create Table Summaries
Here we create the table descriptions and sample questions. These will be tagged against each individual table as the natural-language corpus against which LLM embeddings will be generated. These "dense" embeddings will be used within our dense-retrieval search layer.

#### Methodology
Some background -
- The sparse retrieval layer (using BM25 generated on the complete table) will reduce the search space from 12,000+ documents (tables) to a few hundred (say 200).
- The dense retrieval layer (using LLM embeddings generated on a table summary) will further reduce the search space from 200 to 20.
- This hybrid retrieval engine will allow lexical similarities as well as semantic similarities

Let's get started.

#### 1. Load the CSO dump

In [1]:
from pathlib import Path
import os

root = Path().absolute().parents[1]
os.chdir(str(root))

from src.helpers.json_stat_archive_db import JSONStatArchiveDB

In [2]:
db = JSONStatArchiveDB(compression_level=12)

cso_files = {}
for tid, ds, ts in db.read("artifacts/cso_bkp/cso_archive/jsonstat_archive.sqlite", table_id=None, with_labels=True):
    cso_files[tid] = {
        "data": ds,
        "timestamp": ts,
    }

In [4]:
len(cso_files)

12435

### 2. Create table summaries
We will generate:
- Table Description
- A few synthetic question snippets that mirror how users actually ask the questions that might be relevant to this table

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm_low = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
llm_low.invoke("Hi there!")

AIMessage(content='Hi! How can I help you today? ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run--4f32f703-7d5d-4851-be62-540004a9a230-0', usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}})

In [ ]:
from pydantic import BaseModel, Field
from textwrap import dedent
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed


# Shorten up the JSON for adjusting to Gemini rate limits
def shorten_json(cso_file: dict) -> dict:
    col_ids = cso_file['id']

    for col_id in col_ids:
        val_indices = list(set(cso_file['dimension'][col_id]['category']['index']))[:100]
        val_labels = {}

        for k, v in cso_file['dimension'][col_id]['category']['label'].items():
            if k in val_indices:
                val_labels[k] = v

        cso_file['dimension'][col_id]['category']['index'] = val_indices
        cso_file['dimension'][col_id]['category']['label'] = val_labels

    cso_file['value'] = cso_file['value'][:2]

    return cso_file

class SampleQuestionsModel(BaseModel):
    table_id: str = Field(description="The unique identifier for the CSO API table.")
    sample_questions: list[str] = Field(description="A list of questions to generate synthetic question snippets from the CSO API data.")
    what_this_table_can_answer: str = Field(description="A detailed description of what this table can answer based on the CSO API data.")


# parallelise the above code by using multithreading in jupyter notebook, while using tqdm for progress bar
def fetch_sample_questions(cso_file: dict):
    """Fetch sample questions from the CSO API data."""
    prompt = dedent(
        """\
            # GOAL: You will be given a JSON object containing data from the CSO API. Your task is to generate 10 synthetic question snippets based on the data provided in the JSON object. These questions should mirror how users actually ask questions based on such a data.

            # RESPONSE FORMAT:
            You will return a list of 10 questions / user prompts, where each question / prompt is concise and somewhat relevant to the content of the JSON object. The questions / prompts should be diverse and cover different aspects of the data.
            For example:
            {{
                "table_id": "...",
                "sample_questions": [...],
                "what_this_table_can_answer": "A detailed description of what this table can answer."
            }}

            # TIPS:
            - You may choose to use inexact words or phrases, the same way a user might ask a question.

            # CONTEXT: 
            - table_id: {table_id}
            - json_contents: {output}
        """
    )
    table_id = cso_file['extension']["matrix"]
    
    prompt = prompt.format(
        table_id=table_id,
        output=shorten_json(cso_file)
    )

    response = llm_low.with_structured_output(SampleQuestionsModel).invoke(prompt)
    response_dict = response.model_dump()
    return table_id, response_dict["what_this_table_can_answer"], response_dict["sample_questions"]

cso_data_summaries = []

with ThreadPoolExecutor(max_workers=32) as executor:
    # futures = {executor.submit(fetch_sample_questions, cso_file["data"]): cso_file for cso_file in cso_files.values()}
    futures = {executor.submit(fetch_sample_questions, cso_file["data"]): cso_file for cso_file in x[:50]}

    for future in tqdm(as_completed(futures), total=len(futures)):
        table_id, description, sample_questions = future.result()
        cso_data_summaries.append({
            "id": table_id,
            "description": description,
            "sample_questions": sample_questions,
        })

I0000 00:00:1755709068.228166  959381 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
100%|██████████| 50/50 [00:04<00:00, 11.12it/s]


In [ ]:
import json

# checkpoint: save table summaries
with open("artifacts/cso_smry/summaries.jsonl", "w") as f:
    for item in cso_data_summaries:
        f.write(json.dumps(item) + "\n")

### 3. Create Gemini Embeddings and store in FAISS DB

In [33]:
# load table summaries
with open("artifacts/cso_smry/summaries.jsonl", "r") as f:
    cso_data_summaries = [json.loads(line) for line in f]

In [34]:
len(cso_data_summaries)

12435

In [ ]:
text_chunks = [
    summary["description"] + "\n\n" + "\n".join(summary["sample_questions"]) for summary in cso_data_summaries
]
len(text_chunks)

12435

In [132]:
text_chunks

["This table provides data on Private Dwellings in Permanent Housing Units, broken down by location (towns by size and state) and the period in which they were built. The data is from the 2006 census.\n\nHow many private dwellings were there in Dublin and Suburbs in 2006?\nWhat's the number of private dwellings built before 1919?\nIn what year were the most private dwellings built?\nCan I see the number of private dwellings in Galway City?\nHow many private dwellings were built between 1919 and 1940?\nWhat is the total number of private dwellings?\nHow many dwellings were built in 2001 or later?\nWhat is the number of private dwellings in each town?\nCan you provide data on private dwellings in Ennis?\nShow me the private dwellings in the 'Remainder of country' area.",
 'This table provides data on the population usually resident and present in the state, including breakdowns by age group, sex, ethnic or cultural background, and marital status. The data is from the 2006 census.\n\nHow 

In [41]:
cso_data_summaries[0]

{'id': 'C0620',
 'description': 'This table provides data on Private Dwellings in Permanent Housing Units, broken down by location (towns by size and state) and the period in which they were built. The data is from the 2006 census.',
 'sample_questions': ['How many private dwellings were there in Dublin and Suburbs in 2006?',
  "What's the number of private dwellings built before 1919?",
  'In what year were the most private dwellings built?',
  'Can I see the number of private dwellings in Galway City?',
  'How many private dwellings were built between 1919 and 1940?',
  'What is the total number of private dwellings?',
  'How many dwellings were built in 2001 or later?',
  'What is the number of private dwellings in each town?',
  'Can you provide data on private dwellings in Ennis?',
  "Show me the private dwellings in the 'Remainder of country' area."]}

In [ ]:
import faiss
from google.genai import types
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_google_genai import GoogleGenerativeAIEmbeddings


output_dimensionality = 3072
embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001",
    task_type="semantic_similarity",
    config=types.EmbedContentConfig(
            output_dimensionality=3072,
            task_type="SEMANTIC_SIMILARITY",
        )
)
index = faiss.IndexFlatL2(output_dimensionality)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
documents = [
    Document(
        page_content=cso_data_summary["description"] + "\n\n" + "\n".join(cso_data_summary["sample_questions"]),
        metadata={"id": cso_data_summary["id"]}
    )
    for cso_data_summary in cso_data_summaries
]
ids = [cso_data_summary["id"] for cso_data_summary in cso_data_summaries]

vector_store.add_documents(
    documents=documents,
    ids=ids,
    metadata=[{"id": cso_data_summary["id"]} for cso_data_summary in cso_data_summaries]
)

['C0620',
 'C0503',
 'C0615',
 'C0505',
 'C0510',
 'C0614',
 'C0618',
 'C0616',
 'C0619',
 'C0506',
 'B1054',
 'B1053',
 'C0617',
 'C0504',
 'A0413',
 'C0623',
 'A0411',
 'B1068',
 'C0511',
 'C0507',
 'B1055',
 'B1062',
 'C0508',
 'C0612',
 'C0622',
 'B1059',
 'C0624',
 'C0621',
 'C0611',
 'B1063',
 'B1057',
 'C0625',
 'C0626',
 'B1067',
 'B1056',
 'B1058',
 'C0515',
 'C0627',
 'B1064',
 'C0513',
 'B0203',
 'C0509',
 'B1060',
 'A0424',
 'B0204',
 'C0613',
 'C0512',
 'B1128',
 'B1115',
 'C0520',
 'C0517',
 'B1065',
 'C0637',
 'B1061',
 'B1119',
 'C0514',
 'B1125',
 'C0516',
 'B1109',
 'C0704',
 'C0522',
 'C0631',
 'B1105',
 'C0719',
 'C0519',
 'B1114',
 'B1127',
 'B1108',
 'C0518',
 'C0102',
 'C0628',
 'B1102',
 'B1104',
 'C0701',
 'C0706',
 'C0630',
 'A0429',
 'C0634',
 'C0711',
 'C0727',
 'C0633',
 'B1113',
 'C0721',
 'B1110',
 'C0702',
 'B1116',
 'C0733',
 'C0205',
 'C0636',
 'B1123',
 'C0638',
 'C0729',
 'C0731',
 'C0720',
 'B1101',
 'C0710',
 'C0204',
 'C0109',
 'B1103',
 'C0713',


In [ ]:
vector_store.save_local("artifacts/cso_smry/faiss_index")

In [ ]:
# Example usage to load a FAISS index and perform a similarity search
new_vector_store = FAISS.load_local(
    "artifacts/cso_smry/faiss_index", embeddings, allow_dangerous_deserialization=True
)

results = new_vector_store.similarity_search(
    query="Give some statistics on english conversing population",
    k=10,
    # allow_dangerous_deserialization=True
)
results

[Document(id='SAP2022T2T6SA', metadata={'source': 'user_input'}, page_content='This table provides data on speakers of foreign languages and their ability to speak English, broken down by different areas and for the year 2022.\n\nHow many people speak a foreign language very well?\nWhat is the total number of foreign language speakers?\nWhat is the ability to speak English for each area?\nCan you show the number of people who speak a foreign language not at all?\nWhat is the ability to speak English for the year 2022?\nHow many people speak a foreign language well?\nShow me the number of speakers of foreign languages by area.\nWhat is the number of people who speak a foreign language not well?\nHow does the ability to speak English vary across different areas?\nWhat areas have the highest number of foreign language speakers who speak English very well?'),
 Document(id='SAP2011T2T6PROV', metadata={'source': 'user_input'}, page_content='This table provides data on speakers of foreign lan

In [131]:
results[0].model_dump()

{'id': 'SAP2022T2T6SA',
 'metadata': {'source': 'user_input'},
 'page_content': 'This table provides data on speakers of foreign languages and their ability to speak English, broken down by different areas and for the year 2022.\n\nHow many people speak a foreign language very well?\nWhat is the total number of foreign language speakers?\nWhat is the ability to speak English for each area?\nCan you show the number of people who speak a foreign language not at all?\nWhat is the ability to speak English for the year 2022?\nHow many people speak a foreign language well?\nShow me the number of speakers of foreign languages by area.\nWhat is the number of people who speak a foreign language not well?\nHow does the ability to speak English vary across different areas?\nWhat areas have the highest number of foreign language speakers who speak English very well?',
 'type': 'Document'}